<a href="https://colab.research.google.com/github/ah20776/CE807---Assignment/blob/main/CE807_AssignmentTest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer

#try
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Importing libraries

from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from tqdm import tqdm

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import nltk
nltk.download('wordnet')
import pandas as pd
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
# Load news data set
# remove meta data headers footers and quotes from news dataset
dataset = fetch_20newsgroups(shuffle=True,
                            random_state=32,
                            remove=('headers', 'footers', 'qutes'))

#
dataset_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=32, remove=('headers', 'footers', 'qutes'))
dataset_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=32, remove=('headers', 'footers', 'qutes'))
# Check the names of the categories
print(dataset.target_names)
print(dataset_train.target_names)
print(dataset_test.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.

In [25]:
# put your data into a dataframe
news_df_train = pd.DataFrame({'News': dataset_train.data,
                       'Target': dataset_train.target})
news_df_test = pd.DataFrame({'News': dataset_test.data,
                       'Target': dataset_test.target})

# get dimensions of data 
print(news_df_train.shape)
print(news_df_train.head())

print(news_df_test.shape)
print(news_df_test.head())

(11314, 2)
                                                News  Target
0  The real question here in my opinion is what M...       4
1  Please could someone in the US give me the cur...       4
2  Can somebody please help me with information a...      12
3  In article <2077@rwing.UUCP> pat@rwing.UUCP (P...      16
4  From article <1pq6i2$a1f@news.ysu.edu>, by ak2...       7
(7532, 2)
                                                News  Target
0  Radio Shack stores sell them here in the State...      12
1  Has anyone else observed this behaviour and if...       2
2  D. Andrew Byler (db7n+@andrew.cmu.edu) wrote:\...      15
3  Again I assume this is not just flame bait by ...      10
4  In article <1993Apr21.100149.1501@rtsg.mot.com...       7


In [26]:
#clean text data
# remove non alphabetic characters
# remove stopwords and lemmatize

def clean_text(sentence):
    # remove non alphabetic sequences
    pattern = re.compile(r'[^a-z]+')
    sentence = sentence.lower()
    sentence = pattern.sub(' ', sentence).strip()
    
    # Tokenize
    word_list = word_tokenize(sentence)
    
    # stop words
    stopwords_list = set(stopwords.words('english'))
    # puctuation
    #punct = set(string.punctuation)
    
    # remove stop words
    word_list = [word for word in word_list if word not in stopwords_list]
    # remove very small words, length < 3
    # they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 2]

    # remove punctuation
    #word_list = [word for word in word_list if word not in punct]
    
    # stemming
    # ps  = PorterStemmer()
    # word_list = [ps.stem(word) for word in word_list]
    
    # lemmatize
    lemma = WordNetLemmatizer()
    word_list = [lemma.lemmatize(word) for word in word_list]
    # list to sentence
    sentence = ' '.join(word_list)
    
    return sentence

# we'll use tqdm to monitor progress of data cleaning process
# create tqdm for pandas
tqdm.pandas()
# clean text data

news_df_train['News'] = news_df_train['News'].progress_apply(lambda x: clean_text(str(x)))
print(news_df_train.head())

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 11314/11314 [00:21<00:00, 520.45it/s]

                                                News  Target
0  real question opinion motorola processor runni...       4
1  please could someone give current street price...       4
2  somebody please help information american magn...      12
3  article rwing uucp pat rwing uucp pat myrto wr...      16
4  article news ysu edu yfn ysu edu john daker cu...       7


In [35]:
news_NB = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

news_NB = news_NB.fit(news_df_train.News, news_df_train.Target)

In [37]:
# Performance of NB Classifier
predicted_news_NB = news_NB.predict(news_df_test.News)
np.mean(predicted_news_NB == news_df_test.Target)

0.7446893255443441

In [38]:
# Training Support Vector Machines - SVM and calculating its performance

news_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42))])

news_svm = news_svm.fit(news_df_train.News, news_df_train.Target)
predicted_news_svm = news_svm.predict(news_df_test.News)
np.mean(predicted_news_svm == news_df_test.Target)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.7547796070100903

In [ ]:
# Grid Search
# Here, we are creating a list of parameters for which we would like to do performance tuning.
# All the parameters name start with the classifier name (remember the arbitrary name we gave).
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

# Next, we create an instance of the grid search by passing the classifier, parameters
# and n_jobs=-1 which tells to use multiple cores from user machine.

# with Naive Bayes

gs_news_NB = GridSearchCV(news_NB, parameters, n_jobs=-1)
gs_news_NB = gs_news_NB.fit(news_df_train.News, news_df_train.Target)